In [ ]:
import numpy as np
import pandas as pd
from typing import Tuple, Optional

# ----------------------------
# 1) PERIOD (n_lines) ESTIMATION
# ----------------------------

def _acf(y: np.ndarray, max_lag: int) -> np.ndarray: # ->basicamente isso aqui calcula a galera com maior correlação e deixa em uma só linha
    """Biased ACF up to max_lag (lag 0..max_lag). NaNs are linearly interpolated first."""
    y = pd.Series(y).interpolate(limit_direction="both").to_numpy()
    y = y - np.nanmean(y)
    n = len(y)
    acf_vals = np.empty(max_lag + 1)
    denom = np.dot(y, y) + 1e-12
    for lag in range(max_lag + 1):
        acf_vals[lag] = np.dot(y[: n - lag], y[lag:]) / denom
    return acf_vals

def _fft_period(y: np.ndarray, min_period: int, max_period: int) -> Optional[int]: # isso aqui faz a mesma coisa só que com fourrier 
    """FFT-based dominant period in [min_period, max_period], None if not found."""
    y = pd.Series(y).interpolate(limit_direction="both").to_numpy()
    y = y - np.mean(y)
    n = len(y)
    if n < 4:
        return None
    # Real FFT spectrum
    spec = np.fft.rfft(y)
    freqs = np.fft.rfftfreq(n, d=1.0)  # assume unit sampling
    # Exclude DC
    mask = freqs > 0
    freqs = freqs[mask]
    power = (spec[mask].real**2 + spec[mask].imag**2)
    # Convert freq -> period
    periods = np.round(1.0 / freqs).astype(int)
    # Keep only within bounds
    sel = (periods >= min_period) & (periods <= max_period)
    if not np.any(sel):
        return None
    # Aggregate power by period (many freqs can map to same rounded period)
    dfp = pd.DataFrame({"period": periods[sel], "power": power[sel]})
    top = dfp.groupby("period", as_index=False)["power"].sum().sort_values("power", ascending=False)
    return int(top["period"].iloc[0]) if len(top) else None

def estimate_period(
    y: np.ndarray,
    min_period: int = 4,
    max_period: Optional[int] = None
) -> int:
    """
    Pick period (n_lines) automatically using ACF peak with FFT fallback.
    """
    y = np.asarray(y, dtype=float)
    n = len(y)
    if max_period is None:
        max_period = max(7, min(n // 4, 1000))  # sensible cap

    if n < min_period * 2:
        # too short — just return something small
        return max(min_period, min(n, 8))

    acf_vals = _acf(y, max_period)
    # Ignore lag 0; pick the best lag in [min_period, max_period]
    candidate_lags = np.arange(min_period, max_period + 1)
    best_lag = candidate_lags[np.argmax(acf_vals[min_period: max_period + 1])]

    # FFT fallback check: if ACF peak is weak, try FFT suggestion
    acf_strength = acf_vals[best_lag]
    fft_suggestion = _fft_period(y, min_period, max_period)
    if fft_suggestion is not None:
        if acf_strength < 0.15:  # weak ACF; trust FFT
            return int(fft_suggestion)
        # If both agree closely, prefer the smaller (more stable) period
        if abs(fft_suggestion - best_lag) <= 2:
            return int(min(fft_suggestion, best_lag))
    return int(best_lag)

# ---------------------------------
# 2) FOLDING (PERIODIC TEMPORAL MATRIX)
# ---------------------------------

def fold_series_to_matrix(y: np.ndarray, period: int) -> Tuple[np.ndarray, int]: # basicamente isso cria a matriz temporal 
    """
    Fold 1D series into a (n_blocks, period) matrix. 
    Pads the last block with NaN if needed.
    Returns (M, original_len).
    """
    y = np.asarray(y, dtype=float)
    n = len(y)
    n_blocks = int(np.ceil(n / period)) #n de coluans
    pad_len = n_blocks * period - n
    if pad_len > 0:
        y = np.concatenate([y, np.full(pad_len, np.nan)])
    M = y.reshape(n_blocks, period)
    return M, n

def unfold_matrix_to_series(M: np.ndarray, original_len: int) -> np.ndarray: # volta pro formato original
    """Inverse of fold: row-wise flatten and trim to original length."""
    y = M.reshape(-1)
    return y[:original_len]

# ----------------------------
# 3) SVD + RANK SELECTION
# ----------------------------

def svd_rank(M_filled: np.ndarray, energy: float = 0.9) -> Tuple[np.ndarray, np.ndarray, np.ndarray, int]: # faz o svd na tora e escolhe o r com a soma dos valore sisnuglares so deus sabe pq
    """
    Compute SVD and choose rank r by cumulative explained 'energy' (sum of singular values).
    """
    U, s, Vt = np.linalg.svd(M_filled, full_matrices=False)
    cum = np.cumsum(s) / (np.sum(s) + 1e-12)
    r = int(np.searchsorted(cum, energy) + 1)
    r = max(1, min(r, min(M_filled.shape)))
    return U, s, Vt, r

# -------------------------------------------
# 4) KNN IMPUTE USING ROW EMBEDDINGS FROM SVD
# -------------------------------------------

def _warm_start_fill(M: np.ndarray) -> np.ndarray: # coloca mediana em tudo e vapo só pra começar
    """Column-wise median fill as a stable warm start."""
    M_filled = M.copy()
    col_medians = np.nanmedian(M_filled, axis=0)
    # If an entire column is NaN, fallback to global median
    if np.any(np.isnan(col_medians)):
        global_med = np.nanmedian(M_filled)
        col_medians = np.where(np.isnan(col_medians), global_med, col_medians)
    inds = np.where(np.isnan(M_filled))
    M_filled[inds] = np.take(col_medians, inds[1])
    return M_filled

def impute_with_knn_in_latent( 
    M: np.ndarray,
    k: int = 5,
    energy: float = 0.9,
    allow_future: bool = True
) -> np.ndarray:
    """
    Impute NaNs in M by KNN in SVD latent space (rows ≈ cycles/weeks). que porra eh latent space
    For each missing cell (i,j), find k nearest rows to row i in latent space
    among those with M[row, j] observed (and optionally row < i).
    """
    M_filled0 = _warm_start_fill(M)
    U, s, Vt, r = svd_rank(M_filled0, energy=energy)
    # Row embeddings (T x r): U_r * S_r
    Z = U[:, :r] * s[:r]  # broadcasting: each column of U scaled by s

    M_imp = M.copy()
    T, P = M.shape
    eps = 1e-8

    # Precompute which rows have each column observed
    observed_mask = ~np.isnan(M)
    for i in range(T):
        # indices (columns) that are missing in row i
        miss_cols = np.where(~observed_mask[i])[0]
        if len(miss_cols) == 0:
            continue
        zi = Z[i]

        # Candidate rows for neighbors (global, filtered below per col)
        if allow_future:
            candidate_rows_global = np.arange(T)
        else:
            candidate_rows_global = np.arange(0, i)  # only past

        if len(candidate_rows_global) == 0:
            # If we can't use past (i==0), allow future just for this row:
            candidate_rows_global = np.arange(T)

        # Distances in latent space to all candidates
        Zc = Z[candidate_rows_global]
        dists = np.linalg.norm(Zc - zi[None, :], axis=1)
        dists = dists + eps  # avoid zero

        for j in miss_cols:
            # keep only candidates that have this column observed
            obs_rows = candidate_rows_global[observed_mask[candidate_rows_global, j]]
            if len(obs_rows) == 0:
                # fall back to warm-start value if nothing observed
                M_imp[i, j] = M_filled0[i, j]
                continue

            # distances for those rows
            d = np.linalg.norm(Z[obs_rows] - zi[None, :], axis=1) + eps
            # k nearest
            if len(d) > k:
                idx = np.argpartition(d, k)[:k]
                nn_rows = obs_rows[idx]
                d = d[idx]
            else:
                nn_rows = obs_rows

            w = 1.0 / d  # inverse-distance weights
            vals = M[nn_rows, j]
            # safety: if still NaN (shouldn't happen), drop them
            ok = ~np.isnan(vals)
            if not np.any(ok):
                M_imp[i, j] = M_filled0[i, j]
            else:
                vals = vals[ok]
                w = w[ok]
                M_imp[i, j] = np.sum(w * vals) / np.sum(w)

    return M_imp

# ----------------------------
# 5) MAIN PIPELINE
# ----------------------------

def impute_throughput_svd_knn(
    df: pd.DataFrame,
    col: str = "throughput_bps",
    min_period: int = 4,
    max_period: Optional[int] = None,
    energy: float = 0.9,
    k: int = 5,
    allow_future: bool = True
) -> Tuple[pd.Series, dict]:
    """
    Full pipeline:
    1) auto period detection -> n_lines (period)
    2) fold into (n_blocks, period) matrix
    3) SVD -> latent row embeddings
    4) KNN in latent space to impute NaNs
    5) unfold back to 1D series
    Returns (imputed_series, diagnostics).
    """
    if col not in df.columns:
        raise ValueError(f"Column '{col}' not found in df.")

    y = df[col].to_numpy(dtype=float)
    original_index = df.index

    # Detect period (n_lines)
    period = estimate_period(y, min_period=min_period, max_period=max_period)

    # Fold
    M, orig_len = fold_series_to_matrix(y, period=period)

    # Impute in latent KNN space
    M_imp = impute_with_knn_in_latent(M, k=k, energy=energy, allow_future=allow_future)

    # Unfold
    y_imp = unfold_matrix_to_series(M_imp, original_len=orig_len)

    print(y_imp)

    diagnostics = {
        "period_estimated": period,
        "matrix_shape": M.shape,
        "rank_energy_target": energy,
        "k_neighbors": k,
        "allow_future": allow_future,
    }
    return pd.Series(y_imp, index=original_index, name=f"{col}_imputed"), diagnostics



In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Assume you imported/improved the impute_throughput_svd_knn from the prototype code
# from svd_knn_pipeline import impute_throughput_svd_knn   # example import

BASE_DIR = "./test-data"

# --------------------------
# Inject Missing Data
# --------------------------
def introduce_missing_data(df, missing_rate, seed=42):
    rng = np.random.default_rng(seed)
    df_missing = df.copy()
    mask = rng.random(len(df_missing)) < missing_rate
    df_missing.loc[mask, "throughput_bps"] = np.nan
    print(f"Introduced {missing_rate * 100}% missing data in {len(mask)} rows.")
    return df_missing, mask   # return also the mask to evaluate later

# --------------------------
# Evaluation Function
# --------------------------
def evaluate_imputation(mask_missing, df_true, df_imputed, method, file, rate, results):
    y_true = df_true.loc[mask_missing, "throughput_bps"].values
    y_pred = df_imputed.loc[mask_missing, "throughput_bps_imputed"].values

    if len(y_true) > 0:
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        nrmse = rmse / (y_true.max() - y_true.min())  # normalized by range
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

        results.append({
            "file": file,
            "rate": rate,
            "method": method,
            "rmse": rmse,
            "nrmse": nrmse,
            "mae": mae,
            "r2": r2,
        })

    return results

# --------------------------
# Main Loop
# --------------------------
datasets_missing = {}
results = []

for file in os.listdir(BASE_DIR):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(BASE_DIR, file))
        base_key = file.removesuffix(".throughput.csv")
        datasets_missing[base_key] = {}

        for rate in [0.1, 0.2, 0.3, 0.4]:
            # introduce missing
            df_missing, mask_missing = introduce_missing_data(df, missing_rate=rate, seed=42)
            rate_key = f"{int(rate * 100)}"
            datasets_missing[base_key][rate_key] = df_missing

            # --------------------------
            # Apply your SVD+KNN pipeline
            # --------------------------
            df_imputed, diag = impute_throughput_svd_knn(
                df_missing,
                col="throughput_bps",
                min_period=24,   # adjust for your sampling rate
                max_period=1000, # search window for period detection
                energy=0.9,
                k=5,
                allow_future=True
            )

            # Merge back into DataFrame with same structure
            df_result = df_missing.copy()
            df_result["throughput_bps_imputed"] = df_imputed.values

            # --------------------------
            # Evaluate
            # --------------------------
            results = evaluate_imputation(
                mask_missing, 
                df, df_result, 
                method="Hankel+SVD+KNN", 
                file=file, 
                rate=rate, 
                results=results
            )

# --------------------------
# Save Results
# --------------------------
df_results = pd.DataFrame(results)
df_results.to_csv("results.csv", index=False)
print("Resultados salvos em results.csv")
print(df_results.head())


Introduced 10.0% missing data in 6717 rows.
[ 6291904.17111111  7446471.12444444 10027996.31194444 ...
 18689784.75972222 11790824.49611111 13256918.35777778]
Introduced 20.0% missing data in 6717 rows.
[ 6291904.17111111  7446471.12444444 10027996.31194444 ...
 18689784.75972222 11790824.49611111 13256918.35777778]
Introduced 30.0% missing data in 6717 rows.
[ 6291904.17111111  7446471.12444444 10027996.31194444 ...
 18689784.75972222 11790824.49611111 13256918.35777778]
Introduced 40.0% missing data in 6717 rows.
[ 6291904.17111111  7446471.12444444 10027996.31194444 ...
 18689784.75972222 11790824.49611111 13256918.35777778]
Introduced 10.0% missing data in 6717 rows.
[8276379.73805556 6299608.46222222 7835900.39555556 ... 8620657.81083333
 6094621.17694444 6457880.36583333]
Introduced 20.0% missing data in 6717 rows.
[8276379.73805556 6299608.46222222 7835900.39555556 ... 8620657.81083333
 6094621.17694444 6457880.36583333]
Introduced 30.0% missing data in 6717 rows.
[8276379.73805